Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = '/home/panas/Desktop/udacityData/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

$L' = L + \beta || w ||^2_2$

# Logistic regression model

In [6]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    
    regularizer = tf.nn.l2_loss(weights) 
    loss = loss + beta*regularizer

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
num_steps = 1000

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
    tf.initialize_all_variables().run() #original code: global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 20.313641
Training accuracy: 14.0%
Validation accuracy: 17.8%
Loss at step 100: 4.738130
Training accuracy: 71.7%
Validation accuracy: 71.7%
Loss at step 200: 3.916709
Training accuracy: 74.9%
Validation accuracy: 74.4%
Loss at step 300: 3.367466
Training accuracy: 76.6%
Validation accuracy: 75.8%
Loss at step 400: 2.944789
Training accuracy: 77.7%
Validation accuracy: 76.6%
Loss at step 500: 2.602464
Training accuracy: 78.5%
Validation accuracy: 76.8%
Loss at step 600: 2.318432
Training accuracy: 79.4%
Validation accuracy: 77.2%
Loss at step 700: 2.079281
Training accuracy: 80.2%
Validation accuracy: 77.6%
Loss at step 800: 1.876033
Training accuracy: 80.9%
Validation accuracy: 78.0%
Loss at step 900: 1.702250
Training accuracy: 81.5%
Validation accuracy: 78.3%
Test accuracy: 85.9%


# Neural Network

In [8]:
batch_size = 128
beta =0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # regularizer
  regularizer = beta*tf.nn.l2_loss(weights)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [9]:
num_steps = 5000

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run() #global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.840139
Minibatch accuracy: 7.8%
Validation accuracy: 15.0%
Minibatch loss at step 500: 1.395745
Minibatch accuracy: 73.4%
Validation accuracy: 76.2%
Minibatch loss at step 1000: 1.720167
Minibatch accuracy: 77.3%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 0.997231
Minibatch accuracy: 79.7%
Validation accuracy: 77.5%
Minibatch loss at step 2000: 0.813703
Minibatch accuracy: 78.1%
Validation accuracy: 78.0%
Minibatch loss at step 2500: 0.981871
Minibatch accuracy: 75.0%
Validation accuracy: 78.5%
Minibatch loss at step 3000: 1.013514
Minibatch accuracy: 78.1%
Validation accuracy: 78.4%
Minibatch loss at step 3500: 0.656795
Minibatch accuracy: 84.4%
Validation accuracy: 79.3%
Minibatch loss at step 4000: 0.606673
Minibatch accuracy: 82.8%
Validation accuracy: 80.2%
Minibatch loss at step 4500: 1.004496
Minibatch accuracy: 78.9%
Validation accuracy: 80.3%
Test accuracy: 87.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [12]:
num_steps = 5000
train_dataset2 = train_dataset[:500,:]
train_labels2 = train_labels[:500,:]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run() #global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels2.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset2[offset:(offset + batch_size), :]
    batch_labels = train_labels2[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.693468
Minibatch accuracy: 9.4%
Validation accuracy: 10.3%
Minibatch loss at step 500: 0.125503
Minibatch accuracy: 98.4%
Validation accuracy: 67.0%
Minibatch loss at step 1000: 0.014652
Minibatch accuracy: 100.0%
Validation accuracy: 67.7%
Minibatch loss at step 1500: 0.007606
Minibatch accuracy: 100.0%
Validation accuracy: 68.2%
Minibatch loss at step 2000: 0.005543
Minibatch accuracy: 100.0%
Validation accuracy: 68.6%
Minibatch loss at step 2500: 0.003829
Minibatch accuracy: 100.0%
Validation accuracy: 68.7%
Minibatch loss at step 3000: 0.003104
Minibatch accuracy: 100.0%
Validation accuracy: 68.8%
Minibatch loss at step 3500: 0.002315
Minibatch accuracy: 100.0%
Validation accuracy: 68.9%
Minibatch loss at step 4000: 0.001971
Minibatch accuracy: 100.0%
Validation accuracy: 69.1%
Minibatch loss at step 4500: 0.001766
Minibatch accuracy: 100.0%
Validation accuracy: 69.3%
Test accuracy: 75.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [36]:
num_nodes= 1024
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weightsL1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biasesL1 = tf.Variable(tf.zeros([num_nodes]))
    weightsL2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biasesL2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logitsL1 = tf.matmul(tf_train_dataset, weightsL1) + biasesL1
    RELU_Hidden_Layer= tf.nn.relu(logitsL1)
    
    # RELUs Dropout
    keep_prob = tf.placeholder("float")
    RELU_Hidden_Layer_Dropout = tf.nn.dropout(RELU_Hidden_Layer, keep_prob)
    
    logitsL2 = tf.matmul(RELU_Hidden_Layer_Dropout, weightsL2) + biasesL2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logitsL2, tf_train_labels))
    
    #Regularizer
    regularizer = tf.nn.l2_loss(weightsL1) + tf.nn.l2_loss(weightsL2)
    loss = loss + beta*regularizer

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training
    train_prediction = tf.nn.softmax(logitsL2)
    
    # Predictions for validation 
    logitsL1 = tf.matmul(tf_valid_dataset, weightsL1) + biasesL1
    RELU_Hidden_Layer= tf.nn.relu(logitsL1)
    logitsL2 = tf.matmul(RELU_Hidden_Layer, weightsL2) + biasesL2
       
    valid_prediction = tf.nn.softmax(logitsL2)
    
    # Predictions for the test data.
    logitsL1 = tf.matmul(tf_test_dataset, weightsL1) + biasesL1
    RELU_Hidden_Layer = tf.nn.relu(logitsL1)
    logitsL2= tf.matmul(RELU_Hidden_Layer, weightsL2) + biasesL2
        
    test_prediction = tf.nn.softmax(logitsL2)

In [37]:
num_steps = 5000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run() #global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 745.329224
Minibatch accuracy: 10.2%
Validation accuracy: 26.1%
Minibatch loss at step 500: 199.854538
Minibatch accuracy: 71.9%
Validation accuracy: 78.0%
Minibatch loss at step 1000: 123.899940
Minibatch accuracy: 67.2%
Validation accuracy: 79.8%
Minibatch loss at step 1500: 69.217903
Minibatch accuracy: 79.7%
Validation accuracy: 82.7%
Minibatch loss at step 2000: 41.350250
Minibatch accuracy: 82.8%
Validation accuracy: 84.0%
Minibatch loss at step 2500: 25.311401
Minibatch accuracy: 78.9%
Validation accuracy: 84.6%
Minibatch loss at step 3000: 15.495343
Minibatch accuracy: 83.6%
Validation accuracy: 85.9%
Minibatch loss at step 3500: 9.505104
Minibatch accuracy: 90.6%
Validation accuracy: 86.5%
Minibatch loss at step 4000: 5.902301
Minibatch accuracy: 89.1%
Validation accuracy: 86.7%
Minibatch loss at step 4500: 3.984066
Minibatch accuracy: 80.5%
Validation accuracy: 87.2%
Test accuracy: 93.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


# Learning rate

In [38]:
num_nodes= 1024
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weightsL1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biasesL1 = tf.Variable(tf.zeros([num_nodes]))
    weightsL2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biasesL2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logitsL1 = tf.matmul(tf_train_dataset, weightsL1) + biasesL1
    RELU_Hidden_Layer= tf.nn.relu(logitsL1)
    
    # RELUs Dropout
    keep_prob = tf.placeholder("float")
    RELU_Hidden_Layer_Dropout = tf.nn.dropout(RELU_Hidden_Layer, keep_prob)
    
    logitsL2 = tf.matmul(RELU_Hidden_Layer_Dropout, weightsL2) + biasesL2

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logitsL2, tf_train_labels))
    
    #Regularizer
    regularizer = tf.nn.l2_loss(weightsL1) + tf.nn.l2_loss(weightsL2)
    loss = loss + beta*regularizer

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training
    train_prediction = tf.nn.softmax(logitsL2)
    
    # Predictions for validation 
    logitsL1 = tf.matmul(tf_valid_dataset, weightsL1) + biasesL1
    RELU_Hidden_Layer= tf.nn.relu(logitsL1)
    logitsL2 = tf.matmul(RELU_Hidden_Layer, weightsL2) + biasesL2
       
    valid_prediction = tf.nn.softmax(logitsL2)
    
    # Predictions for the test data.
    logitsL1 = tf.matmul(tf_test_dataset, weightsL1) + biasesL1
    RELU_Hidden_Layer = tf.nn.relu(logitsL1)
    logitsL2= tf.matmul(RELU_Hidden_Layer, weightsL2) + biasesL2
        
    test_prediction = tf.nn.softmax(logitsL2)

In [39]:
num_steps = 5000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run() #global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 713.894775
Minibatch accuracy: 12.5%
Validation accuracy: 24.6%
Minibatch loss at step 500: 206.285721
Minibatch accuracy: 67.2%
Validation accuracy: 78.7%
Minibatch loss at step 1000: 118.977432
Minibatch accuracy: 75.0%
Validation accuracy: 80.6%
Minibatch loss at step 1500: 69.015808
Minibatch accuracy: 78.1%
Validation accuracy: 83.1%
Minibatch loss at step 2000: 41.855766
Minibatch accuracy: 79.7%
Validation accuracy: 84.1%
Minibatch loss at step 2500: 25.343262
Minibatch accuracy: 77.3%
Validation accuracy: 85.0%
Minibatch loss at step 3000: 15.568450
Minibatch accuracy: 78.1%
Validation accuracy: 86.2%
Minibatch loss at step 3500: 9.600666
Minibatch accuracy: 85.2%
Validation accuracy: 86.4%
Minibatch loss at step 4000: 5.921721
Minibatch accuracy: 88.3%
Validation accuracy: 86.9%
Minibatch loss at step 4500: 4.025702
Minibatch accuracy: 80.5%
Validation accuracy: 87.2%
Test accuracy: 93.2%


# Multiple Hidden Layers


- 5 hidden layers NN

    - RELUs
    - Number of nodes decrease by 50% with each hidden layer that is deeper in the neural net

- Overfitting measures

    - L2 Regularization
        - Learning rate (beta) with exponential decay
    - Dropout

- 10,000 step

In [58]:
import math

batch_size = 128
beta = 0.001

hidden_nodes_L1 = 1024
hidden_nodes_L2 = int(hidden_nodes_L1*0.5)
hidden_nodes_L3 = int(hidden_nodes_L2*np.power(0.5, 2))
hidden_nodes_L4 = int(hidden_nodes_L3*np.power(0.5, 3))


graph = tf.Graph()
with graph.as_default():
    #######################################################################
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #######################################################################
    ## Variables.
    ### Hidden Layer 1
    weightsL1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_L1],stddev=math.sqrt(2.0/(image_size*image_size))))
    biasesL1 = tf.Variable(tf.zeros([hidden_nodes_L1]))
    
    ### Hidden Layer 2
    weightsL2 = tf.Variable(tf.truncated_normal([hidden_nodes_L1, hidden_nodes_L2], stddev=math.sqrt(2.0/hidden_nodes_L1)))
    biasesL2 = tf.Variable(tf.zeros([hidden_nodes_L2]))
    
    ### Hidden Layer 3
    weightsL3 = tf.Variable(tf.truncated_normal([hidden_nodes_L2, hidden_nodes_L3], stddev=math.sqrt(2.0/hidden_nodes_L2)))
    biasesL3 = tf.Variable(tf.zeros([hidden_nodes_L3]))
    
    ### Hidden Layer 4
    weightsL4 = tf.Variable(tf.truncated_normal([hidden_nodes_L3, hidden_nodes_L4], stddev=math.sqrt(2.0/hidden_nodes_L3)))
    biasesL4 = tf.Variable(tf.zeros([hidden_nodes_L4]))
    
    ### Output Layer 5
    weightsL5 = tf.Variable(tf.truncated_normal([hidden_nodes_L4, num_labels], stddev=math.sqrt(2.0/hidden_nodes_L4)))
    biasesL5 = tf.Variable(tf.zeros([num_labels]))
    
    #######################################################################
    
    ## Training computation.
    ### Hidden Layer 1
    logitsL1 = tf.matmul(tf_train_dataset, weightsL1) + biasesL1
    RELU_Hidden_Layer_1 = tf.nn.relu(logitsL1)
    #### RELUs Dropout
    keep_prob = tf.placeholder("float")
    RELU_Hidden_Layer_1_Dropout = tf.nn.dropout(RELU_Hidden_Layer_1, keep_prob)
    
    ### Hidden Layer 2
    logitsL2 = tf.matmul(RELU_Hidden_Layer_1_Dropout, weightsL2) + biasesL2
    RELU_Hidden_Layer_2 = tf.nn.relu(logitsL2)
    #### RELUs Dropout
    RELU_Hidden_Layer_2_Dropout = tf.nn.dropout(RELU_Hidden_Layer_2, keep_prob)
    
    ### Hidden Layer 3
    logitsL3 = tf.matmul(RELU_Hidden_Layer_2_Dropout, weightsL3) + biasesL3
    RELU_Hidden_Layer_3 = tf.nn.relu(logitsL3)
    #### RELUs Dropout
    RELU_Hidden_Layer_3_Dropout = tf.nn.dropout(RELU_Hidden_Layer_3, keep_prob)
    
    ### Hidden Layer 4
    logitsL4 = tf.matmul(RELU_Hidden_Layer_3_Dropout, weightsL4) + biasesL4
    RELU_Hidden_Layer_4 = tf.nn.relu(logitsL4)
    #### RELUs Dropout
    RELU_Hidden_Layer_4_Dropout = tf.nn.dropout(RELU_Hidden_Layer_4, keep_prob)
    #######################################################################
    
    
    # Output layer
    logitsL5 = tf.matmul(RELU_Hidden_Layer_4_Dropout, weightsL5) + biasesL5

    #######################################################################
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logitsL5, tf_train_labels))
    
    #Regularizers
    regularizer = tf.nn.l2_loss(weightsL1) + tf.nn.l2_loss(weightsL2) + \
                  tf.nn.l2_loss(weightsL3) + tf.nn.l2_loss(weightsL4) + \
                  tf.nn.l2_loss(weightsL5)
    loss = loss + beta*regularizer
    #######################################################################
    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    #######################################################################
    
    # Predictions for the training
    train_prediction = tf.nn.softmax(logitsL5)
    
    #######################################################################
    # Predictions for validation 
    ## Hidden Layer 1
    logitsL1 = tf.matmul(tf_valid_dataset, weightsL1) + biasesL1
    RELU_Hidden_Layer_1 = tf.nn.relu(logitsL1)
    
    ## Hidden Layer 2
    logitsL2 = tf.matmul(RELU_Hidden_Layer_1, weightsL2) + biasesL2
    RELU_Hidden_Layer_2 = tf.nn.relu(logitsL2)
    
    ## Hidden Layer 3
    logitsL3 = tf.matmul(RELU_Hidden_Layer_2, weightsL3) + biasesL3
    RELU_Hidden_Layer_3 = tf.nn.relu(logitsL3)
    
    ## Hidden Layer 4
    logitsL4 = tf.matmul(RELU_Hidden_Layer_3, weightsL4) + biasesL4
    RELU_Hidden_Layer_4 = tf.nn.relu(logitsL4)
    
    ## Ouput Layer
    logitsL5 = tf.matmul(RELU_Hidden_Layer_4, weightsL5) + biasesL5
  
    valid_prediction = tf.nn.softmax(logitsL5)
    #######################################################################
    # Predictions for the test data.
    ## Hidden Layer 1
    logitsL1 = tf.matmul(tf_test_dataset, weightsL1) + biasesL1
    RELU_Hidden_Layer_1 = tf.nn.relu(logitsL1)
    
    ## Hidden Layer 2
    logitsL2 = tf.matmul(RELU_Hidden_Layer_1, weightsL2) + biasesL2
    RELU_Hidden_Layer_2 = tf.nn.relu(logitsL2)
    
    ## Hidden Layer 3
    logitsL3 = tf.matmul(RELU_Hidden_Layer_2, weightsL3) + biasesL3
    RELU_Hidden_Layer_3 = tf.nn.relu(logitsL3)
    
    ## Hidden Layer 4
    logitsL4 = tf.matmul(RELU_Hidden_Layer_3, weightsL4) + biasesL4
    RELU_Hidden_Layer_4 = tf.nn.relu(logitsL4)
    
    ## Output Layer 
    logitsL5 = tf.matmul(RELU_Hidden_Layer_4, weightsL5) + biasesL5
        
    test_prediction = tf.nn.softmax(logitsL5)

In [59]:
num_steps = 15000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run() #global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.383162
Minibatch accuracy: 10.2%
Validation accuracy: 11.6%
Minibatch loss at step 500: 2.065448
Minibatch accuracy: 70.3%
Validation accuracy: 81.2%
Minibatch loss at step 1000: 1.493258
Minibatch accuracy: 75.0%
Validation accuracy: 84.7%
Minibatch loss at step 1500: 1.265282
Minibatch accuracy: 75.0%
Validation accuracy: 84.6%
Minibatch loss at step 2000: 1.150429
Minibatch accuracy: 77.3%
Validation accuracy: 85.4%
Minibatch loss at step 2500: 1.448007
Minibatch accuracy: 71.1%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 1.086213
Minibatch accuracy: 71.9%
Validation accuracy: 85.9%
Minibatch loss at step 3500: 1.177711
Minibatch accuracy: 71.1%
Validation accuracy: 84.5%
Minibatch loss at step 4000: 0.955750
Minibatch accuracy: 78.9%
Validation accuracy: 85.5%
Minibatch loss at step 4500: 1.099899
Minibatch accuracy: 74.2%
Validation accuracy: 84.5%
Minibatch loss at step 5000: 1.093378
Minibatch accuracy: 77.3%
Validation accurac